In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from scipy import io
import os
import wandb

from peter_functions import *

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'hidden_size': {
        'values': [16,32,64]
        },
    'batch_size': {
        'values': [128]
        },
    'optimizer': {
        'values': ['adam']
        },
    'epochs': {
        'values': [500]
        },
    'learning_rate': {
        'values': [0.001,0.01]
      }, 
    'chop': {
        'values': [32,64,128]
      }, 
    }

sweep_config['parameters'] = parameters_dict
config = wandb.config

# 변경해야하는 변수 모음

In [ ]:
input_size  = 22
n_class     = 2
proj_name   = "BEEE_short"
file_path = "C:/Users/Peter/Desktop/CGX/data/less_aug"

sweep_id = wandb.sweep(sweep_config, project=proj_name)

In [ ]:
class TextLSTM(nn.Module):
  def __init__(self,hidden_size):
    super(TextLSTM, self).__init__()

    self.lstm_1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
    self.lstm_2 = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
    self.fc_1 = nn.Linear(hidden_size*2, hidden_size*2)
    self.fc_2 = nn.Linear(hidden_size*2, n_class)

  def forward(self, hidden_and_cell_k, hidden_and_cell_a, K_and_A):
    (k, a) = K_and_A

    k.transpose_(0,1)
    k.transpose_(0,2)
    a.transpose_(0,1)
    a.transpose_(0,2)

    outputs1, (h_n1,c_n1) = self.lstm_1(k, hidden_and_cell_k)
    outputs2, (h_n2,c_n2) = self.lstm_2(a, hidden_and_cell_a)

    outputs = torch.cat((outputs1[-1],outputs2[-1]), dim=1)  

    x = self.fc_1(outputs)  # 최종 예측 최종 출력 층
    model = self.fc_2(F.relu(x))
    return model.to(DEVICE)

In [ ]:
def One_agent_run(config=None):
    with wandb.init(config=config):
        config = wandb.config
        k_train, a_train, y_train   = load_mat_file(config.subject_label, config.chop, file_path, 1)
        k_test, a_test, y_test      = load_mat_file(config.subject_label, config.chop, file_path, 2)
        train_DL, val_DL, test_DL, tf_DL   = build_dataset_tf(config.batch_size, k_train.to(DEVICE), a_train.to(DEVICE), y_train.to(DEVICE),k_test.to(DEVICE), a_test.to(DEVICE), y_test.to(DEVICE))
        
        model = TextLSTM(hidden_size=config.hidden_size).to(DEVICE)
        
        Train(model, train_DL, val_DL, config)
        Test (model, test_DL, config)

        Train_tf(model, tf_DL, config = config)
        Test (model, test_DL, config)


In [ ]:
wandb.agent(sweep_id, One_agent_run)